In [ ]:
import pandas as pd 

df_tagged_news = pd.read_csv('./data/ai_news_large_tagged.csv')
df_cate_news = pd.read_csv('./data/ai_news_categorized_fin.csv')


In [322]:
df_tagged_news = df_tagged_news.drop(columns=['Unnamed: 0', 'text_for_keywords', 
                                              'keywords_str', 'Title Entities', 'Abstract Entities',
                                              'URL','Category', 'SubCategory'])


In [323]:
df_cate_news = df_cate_news['category']

In [324]:
df_cate_news = df_cate_news.apply(
    lambda x: "Misc" if x == "Uncategorized" else (x.split('/')[0] if '/' in x else x)
)

In [325]:
df_tagged_news.head()

,News ID,Title,Abstract,keywords,tags
0,N2804,"Google discontinues Clips, the AI-powered came...",While Google was busy showcasing its latest de...,"['latest devices yesterday', 'Google discontin...","['policy/Regulation', 'topic/Safety', 'geo/US']"
1,N96829,AI can help doctors spot brain hemorrhages faster,AI is already capable of discovering medical c...,"['discovering medical conditions', 'doctors sp...","['domain/Search', 'event/GoogleIO', 'topic/Mul..."
2,N118770,Pentagon's draft AI ethics guidelines fight bi...,Tech companies might have trouble establishing...,"['rogue machines Tech', 'machines Tech compani...","['org/Google', 'domain/Technology', 'event/Goo..."
3,N110386,Google Says New AI-Powered Search Update Is 'H...,Google is injecting its search engine with new...,"['Huge Step Forward', 'AI-Powered Search Updat...","['domain/Healthcare', 'topic/Multimodal', 'top..."
4,N83002,Sonar drone helps find a WWII Japanese aircraf...,The late Paul Allen's underwater robotics are ...,"['Paul Allen underwater', 'late Paul Allen', '...","['domain/Transportation', 'topic/Robotics', 'g..."


In [326]:
df_cate_news

0        Incidents & Safety
1                  Research
2       Policy & Regulation
3      Technology & Product
4        Incidents & Safety
               ...         
142                    Misc
143                    Misc
144                    Misc
145       Society & Culture
146    Technology & Product
Name: category, Length: 147, dtype: object

In [327]:
df_ai_news = pd.concat([df_tagged_news, df_cate_news], axis=1)
df_ai_news

,News ID,Title,Abstract,keywords,tags,category
0,N2804,"Google discontinues Clips, the AI-powered came...",While Google was busy showcasing its latest de...,"['latest devices yesterday', 'Google discontin...","['policy/Regulation', 'topic/Safety', 'geo/US']",Incidents & Safety
1,N96829,AI can help doctors spot brain hemorrhages faster,AI is already capable of discovering medical c...,"['discovering medical conditions', 'doctors sp...","['domain/Search', 'event/GoogleIO', 'topic/Mul...",Research
2,N118770,Pentagon's draft AI ethics guidelines fight bi...,Tech companies might have trouble establishing...,"['rogue machines Tech', 'machines Tech compani...","['org/Google', 'domain/Technology', 'event/Goo...",Policy & Regulation
3,N110386,Google Says New AI-Powered Search Update Is 'H...,Google is injecting its search engine with new...,"['Huge Step Forward', 'AI-Powered Search Updat...","['domain/Healthcare', 'topic/Multimodal', 'top...",Technology & Product
4,N83002,Sonar drone helps find a WWII Japanese aircraf...,The late Paul Allen's underwater robotics are ...,"['Paul Allen underwater', 'late Paul Allen', '...","['domain/Transportation', 'topic/Robotics', 'g...",Incidents & Safety
...,...,...,...,...,...,...
142,N51029,Big Tech tackles the drug diversion opioid cri...,"Some 10% of pharmacists, nurses, and anesthesi...","['Big Tech tackles', 'diversion opioid crisis'...","['domain/Healthcare', 'org/Google', 'org/Micro...",Misc
143,N105010,Alphabet's rebooted robotics program starts wi...,For all the advances made by robot companies l...,"['performing assistive tasks', 'Boston Dynamic...","['domain/Robotics', 'org/Google', 'topic/Agent...",Misc
144,N78685,2 Simple Steps to Withstand Google's Constant ...,You must do the best job on the internet at de...,"['Withstand Google Constant', 'Google Constant...","['domain/Fintech', 'biz/Funding', 'event/Googl...",Misc
145,N86909,Wild Animation Shows All the Multi-Planet Syst...,The tiny dots represent all the multi-planet s...,"['Kepler Space Telescope', 'Systems Kepler Dis...","['domain/Search', 'policy/Regulation', 'topic/...",Society & Culture


In [328]:
## Tagging inspection 
from IPython.display import display, HTML

display(HTML(df_ai_news['tags'].to_frame().to_html()))

,tags
0,"['policy/Regulation', 'topic/Safety', 'geo/US']"
1,"['domain/Search', 'event/GoogleIO', 'topic/Multimodal', 'biz/Pricing']"
2,"['org/Google', 'domain/Technology', 'event/GoogleIO', 'topic/Multimodal \n\n\n\nLet me explain the choices:\n\n* **org/Google:** Directly mentioned in the title and content.\n* **domain/Technology:** While not explicitly stated', 'the article focuses on a tech product (AI-powered camera). \n* **event/GoogleIO:** The phrase ""while Google was busy showcasing its latest devices yesterday"" strongly implies a recent event like Google I/O.\n* **topic/Multimodal:** The description of the camera capturing ""something interesting"" hints at a multimodal approach (likely combining vision and potentially audio). \n\n\n\nLet me know if you\'d like to explore other potential tags!']"
3,"['domain/Healthcare', 'topic/Multimodal', 'topic/RAG', 'org/UC Berkeley', 'org/UCSF']"
4,"['domain/Transportation', 'topic/Robotics', 'geo/US', 'biz/Funding', 'topic/Multimodal']"
5,"['domain/Robotics', 'org/NVIDIA', 'model/Genie', 'topic/Multimodal', 'event/GoogleIO']"
6,"['domain/Healthcare', 'domain/Fintech', 'domain/Education', 'domain/Transportation', 'topic/Multimodal', 'topic/Agents', 'biz/Funding', 'org/Amazon']"
7,"['domain/Transportation', 'topic/Multimodal', 'geo/US', 'biz/Funding']"
8,"['domain/Photoshop', 'topic/Multimodal', 'model/Smart Compose']"
9,"['topic/DarkTechnology', 'domain/Robotics', 'domain/Transportation', 'biz/Regulation \n\n\nLet me explain my choices:\n\n* **topic/DarkTechnology:** This directly reflects the article\'s focus on potentially negative consequences of AI.\n* **domain/Robotics:** Drones and self-driving vehicles are heavily linked to robotics.\n* **domain/Transportation:** Self-driving vehicles fall squarely under transportation.\n* **biz/Regulation:** The mention of ""open-ended questions about unintended consequences"" suggests a need for regulatory oversight. \n\n\n\nLet me know if you have any other articles you\'d like me to tag!']"


In [329]:
import pandas as pd
import ast
import html

def clean_tag(tag: str) -> str:
    """Normalize a single tag string."""
    if not isinstance(tag, str):
        return None
    
    # 1. Remove HTML escape sequences (&amp; → &)
    tag = html.unescape(tag.strip())
    
    # 2. Skip tags with backslashes
    if '\\' in tag:
        return None
    
    # 3. Remove anything after newline
    tag = tag.split('\n')[0]
    
    # 4. Skip tags with invalid characters or patterns
    if any(char in tag for char in ['*', ':', '**']):
        return None
    
    # 5. Handle malformed tags with multiple slashes
    if tag.count('/') > 1:
        # Keep only the first two parts (prefix/value)
        parts = tag.split('/')
        if len(parts) >= 2:
            tag = f"{parts[0]}/{parts[1]}"
        else:
            return None
    
    # 6. Normalize spacing (remove spaces inside tags)
    tag = tag.replace(" ", "")
    
    # 7. Fix known misclassifications
    tag_fixes = {
        "biz/Regulation": "policy/Regulation",
        "event/Funding": "biz/Funding", 
        "domain/Multimodal": "topic/Multimodal",
        "event/NA": "event/Unknown",
        "domain/SocialNetwork": "domain/SocialMedia",
        "domain/SpaceExploration": "domain/Space",
        "domain/Multimedia": "domain/Media",
        "domain/ImageEditing": "domain/Media",
        "domain/Photoshop": "domain/Media",
        "domain/FutureWorkplace": "domain/Workplace",
        "domain/ConsumerElectronics": "domain/Technology"
    }
    
    if tag in tag_fixes:
        tag = tag_fixes[tag]
    
    return tag

def process_tags(tag_entry):
    """Process a tag entry (string or list) into a clean list of tags."""
    # Handle cases where tag_entry is a string representation of a list
    if isinstance(tag_entry, str):
        try:
            # Safely evaluate string to list if possible
            tags = ast.literal_eval(tag_entry)
        except (ValueError, SyntaxError):
            # Fallback to splitting by commas if evaluation fails
            tags = tag_entry.split(',')
    else:
        tags = tag_entry
    
    # If tags is not a list, return empty list
    if not isinstance(tags, list):
        return []
    
    # Clean each tag and filter valid ones
    cleaned = []
    seen = set()
    
    for tag in tags:
        if isinstance(tag, str):
            cleaned_tag = clean_tag(tag)
            
            # Skip None tags (those with backslashes)
            if cleaned_tag is None:
                continue
                
            # Filter tags to keep only those matching the controlled vocabulary pattern
            # and ensure each tag is one or two words after the prefix
            if cleaned_tag.startswith(('org/', 'model/', 'domain/', 'topic/', 'event/', 'geo/', 'biz/', 'policy/')):
                # Extract the part after the prefix
                tag_value = cleaned_tag.split('/', 1)[1]
                
                # Check if the tag value is one or two words (no spaces after normalization means single concept)
                # We'll be more flexible here since we removed spaces during normalization
                if tag_value and len(tag_value.strip()) > 0:
                    # Remove duplicates while preserving order
                    if cleaned_tag not in seen:
                        seen.add(cleaned_tag)
                        cleaned.append(cleaned_tag)
    
    return cleaned

def clean_tags(tags_series):
    """Clean a pandas Series of tag entries."""
    return tags_series.apply(process_tags)

# Alternative function for already-processed tag lists
def deduplicate_and_filter_tags(tags):
    """Clean a list of tags by removing duplicates, backslashes, and invalid entries."""
    if not isinstance(tags, list):
        return []
        
    seen = set()
    cleaned = []
    
    for tag in tags:
        if isinstance(tag, str):
            cleaned_tag = clean_tag(tag)
            
            # Skip None tags (those with backslashes)
            if cleaned_tag is None:
                continue
                
            # Remove duplicates while preserving order
            if cleaned_tag not in seen:
                seen.add(cleaned_tag)
                cleaned.append(cleaned_tag)
    
    return cleaned

In [330]:
df_ai_news['cleaned_tags'] = clean_tags(df_ai_news['tags'])
df_ai_news['cleaned_tags'] = df_ai_news['cleaned_tags'].apply(deduplicate_and_filter_tags)

In [331]:
from IPython.display import display, HTML

display(HTML(df_ai_news['cleaned_tags'].to_frame().to_html()))


,cleaned_tags
0,"[policy/Regulation, topic/Safety, geo/US]"
1,"[domain/Search, event/GoogleIO, topic/Multimodal, biz/Pricing]"
2,"[org/Google, domain/Technology, event/GoogleIO, topic/Multimodal]"
3,"[domain/Healthcare, topic/Multimodal, topic/RAG, org/UCBerkeley, org/UCSF]"
4,"[domain/Transportation, topic/Robotics, geo/US, biz/Funding, topic/Multimodal]"
5,"[domain/Robotics, org/NVIDIA, model/Genie, topic/Multimodal, event/GoogleIO]"
6,"[domain/Healthcare, domain/Fintech, domain/Education, domain/Transportation, topic/Multimodal, topic/Agents, biz/Funding, org/Amazon]"
7,"[domain/Transportation, topic/Multimodal, geo/US, biz/Funding]"
8,"[domain/Media, topic/Multimodal, model/SmartCompose]"
9,"[topic/DarkTechnology, domain/Robotics, domain/Transportation, policy/Regulation]"


---

In [270]:
df_ai_news = df_ai_news.drop(columns=['tags'])

In [271]:
df_ai_news.head()

,News ID,Title,Abstract,keywords,category,cleaned_tags
0,N2804,"Google discontinues Clips, the AI-powered came...",While Google was busy showcasing its latest de...,"['latest devices yesterday', 'Google discontin...",Incidents & Safety,"[policy/Regulation, topic/Safety, geo/US]"
1,N96829,AI can help doctors spot brain hemorrhages faster,AI is already capable of discovering medical c...,"['discovering medical conditions', 'doctors sp...",Research,"[domain/Search, event/GoogleIO, topic/Multimod..."
2,N118770,Pentagon's draft AI ethics guidelines fight bi...,Tech companies might have trouble establishing...,"['rogue machines Tech', 'machines Tech compani...",Policy & Regulation,"[org/Google, domain/Technology, event/GoogleIO]"
3,N110386,Google Says New AI-Powered Search Update Is 'H...,Google is injecting its search engine with new...,"['Huge Step Forward', 'AI-Powered Search Updat...",Technology & Product,"[domain/Healthcare, topic/Multimodal, topic/RA..."
4,N83002,Sonar drone helps find a WWII Japanese aircraf...,The late Paul Allen's underwater robotics are ...,"['Paul Allen underwater', 'late Paul Allen', '...",Incidents & Safety,"[domain/Transportation, topic/Robotics, geo/US..."


In [272]:
ai_news_id = df_ai_news['News ID'].tolist()
len(ai_news_id)

147

In [273]:
df_useer = pd.read_csv('./Data/behav_all.csv')

In [274]:
def contains_ai_news(row, ai_news_ids, columns=['History', 'Impressions']):
    """Check if row contains any AI news IDs in specified columns"""
    for col in columns:
        if pd.notna(row[col]) and isinstance(row[col], str):
            try:
                for news_id in row[col].split():
                    # Extract base ID (remove suffix after '-' if it exists)
                    news_id_clean = news_id.split('-')[0] if '-' in news_id else news_id
                    if news_id_clean in ai_news_ids:
                        return True
            except:
                continue
    return False

def filter_ai_news_ids(row, ai_news_ids, columns=['History', 'Impressions']):
    """Remove non-AI news IDs from specified columns, keeping only AI-related ones"""
    for col in columns:
        if pd.notna(row[col]) and isinstance(row[col], str):
            try:
                # Split the news IDs
                news_ids = row[col].split()
                # Keep only AI news IDs
                ai_news_ids_filtered = []
                
                for news_id in news_ids:
                    # Extract base ID (remove suffix after '-' if it exists)
                    news_id_clean = news_id.split('-')[0] if '-' in news_id else news_id
                    if news_id_clean in ai_news_ids:
                        ai_news_ids_filtered.append(news_id)
                
                # Update the row with filtered IDs (join back with spaces)
                row[col] = ' '.join(ai_news_ids_filtered) if ai_news_ids_filtered else ''
            except:
                # If there's an error, set to empty string
                row[col] = ''
    return row

# Convert ai_news_ids to set for faster lookups (if it's not already)
ai_news_ids_set = set(ai_news_id) if not isinstance(ai_news_id, set) else ai_news_id

# Step 1: Filter rows that contain AI news IDs
ai_related_mask = df_user.apply(lambda row: contains_ai_news(row, ai_news_ids_set), axis=1)
ai_behaviors_df = df_user[ai_related_mask].copy()

# Step 2: Clean the filtered dataframe to keep only AI news IDs in History/Impressions columns
ai_behaviors_df = ai_behaviors_df.apply(
    lambda row: filter_ai_news_ids(row, ai_news_ids_set, columns=['History', 'Impressions']), 
    axis=1
)

In [275]:
ai_behaviors_df = ai_behaviors_df.reset_index(drop=True)

In [276]:
ai_behaviors_df[['History', 'Impressions']].value_counts()

History  Impressions
         N59088-0       4551
N9824                   1466
         N31395-0       1116
         N56061-0        783
N52946                   604
                        ... 
N2804    N59088-0          1
N52946   N15124-0          1
N44956   N31395-0          1
N62467   N15124-0          1
N52946   N59088-1          1
Name: count, Length: 69, dtype: int64

In [277]:
df_ai_user = ai_behaviors_df.copy()

In [278]:
df_ai_news

,News ID,Title,Abstract,keywords,category,cleaned_tags
0,N2804,"Google discontinues Clips, the AI-powered came...",While Google was busy showcasing its latest de...,"['latest devices yesterday', 'Google discontin...",Incidents & Safety,"[policy/Regulation, topic/Safety, geo/US]"
1,N96829,AI can help doctors spot brain hemorrhages faster,AI is already capable of discovering medical c...,"['discovering medical conditions', 'doctors sp...",Research,"[domain/Search, event/GoogleIO, topic/Multimod..."
2,N118770,Pentagon's draft AI ethics guidelines fight bi...,Tech companies might have trouble establishing...,"['rogue machines Tech', 'machines Tech compani...",Policy & Regulation,"[org/Google, domain/Technology, event/GoogleIO]"
3,N110386,Google Says New AI-Powered Search Update Is 'H...,Google is injecting its search engine with new...,"['Huge Step Forward', 'AI-Powered Search Updat...",Technology & Product,"[domain/Healthcare, topic/Multimodal, topic/RA..."
4,N83002,Sonar drone helps find a WWII Japanese aircraf...,The late Paul Allen's underwater robotics are ...,"['Paul Allen underwater', 'late Paul Allen', '...",Incidents & Safety,"[domain/Transportation, topic/Robotics, geo/US..."
...,...,...,...,...,...,...
142,N51029,Big Tech tackles the drug diversion opioid cri...,"Some 10% of pharmacists, nurses, and anesthesi...","['Big Tech tackles', 'diversion opioid crisis'...",Misc,"[domain/Healthcare, org/Google, org/Microsoft,..."
143,N105010,Alphabet's rebooted robotics program starts wi...,For all the advances made by robot companies l...,"['performing assistive tasks', 'Boston Dynamic...",Misc,"[domain/Robotics, org/Google, topic/Agents, ev..."
144,N78685,2 Simple Steps to Withstand Google's Constant ...,You must do the best job on the internet at de...,"['Withstand Google Constant', 'Google Constant...",Misc,"[domain/Fintech, biz/Funding, event/GoogleIO, ..."
145,N86909,Wild Animation Shows All the Multi-Planet Syst...,The tiny dots represent all the multi-planet s...,"['Kepler Space Telescope', 'Systems Kepler Dis...",Society & Culture,"[domain/Search, policy/Regulation, topic/RAG]"


---

- Data Preprocessing

In [279]:
ai_news_id = df_ai_news['News ID'].to_list()
ai_news_id[:5]

['N2804', 'N96829', 'N118770', 'N110386', 'N83002']

In [280]:
df_ai_user['Time'] = pd.to_datetime(df_ai_user['Time'], errors='coerce')
df_ai_user.head()

,Impression ID,User ID,Time,History,Impressions
0,18,U11306,2019-11-11 04:29:18,,N56061-0
1,22,U80798,2019-11-09 14:23:59,N52946,
2,32,U6744,2019-11-12 12:21:35,N9824,
3,35,U78244,2019-11-12 12:58:56,N44956,
4,54,U2579,2019-11-13 06:58:59,,N59088-0


In [281]:
len(ai_news_id)

147

In [282]:
df_ai_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10090 entries, 0 to 10089
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Impression ID  10090 non-null  int64         
 1   User ID        10090 non-null  object        
 2   Time           10090 non-null  datetime64[ns]
 3   History        9910 non-null   object        
 4   Impressions    10090 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 394.3+ KB


In [283]:
## cold start users
df_ai_user.isna().sum(), len(df_ai_user)

(Impression ID      0
 User ID            0
 Time               0
 History          180
 Impressions        0
 dtype: int64,
 10090)

In [284]:
df_ai_user['History'] = df_ai_user['History'].str.split(" ")
df_ai_user

,Impression ID,User ID,Time,History,Impressions
0,18,U11306,2019-11-11 04:29:18,[],N56061-0
1,22,U80798,2019-11-09 14:23:59,[N52946],
2,32,U6744,2019-11-12 12:21:35,[N9824],
3,35,U78244,2019-11-12 12:58:56,[N44956],
4,54,U2579,2019-11-13 06:58:59,[],N59088-0
...,...,...,...,...,...
10085,156909,U58715,2019-11-11 04:20:16,[],N56061-0
10086,156923,U62649,2019-11-13 07:12:20,[],N59088-0
10087,156945,U31444,2019-11-13 06:34:19,[],N59088-0
10088,156956,U66493,2019-11-13 12:02:07,[N52946],


In [286]:
max_lengtth = 5

def pad_history(history, max_length):
    if len(history) > max_length:
        return history[:max_length]
    else:
        return history + ["0"] * (max_length - len(history))

df_ai_user['History'] = df_ai_user['History'].apply(
    lambda x: x if isinstance(x, list) and len(x) > 0 and all(x) else ["0"] * max_lengtth
)
df_ai_user['History'] = df_ai_user['History'].apply(lambda x: pad_history(x, max_lengtth))
df_ai_user

,Impression ID,User ID,Time,History,Impressions
0,18,U11306,2019-11-11 04:29:18,"[0, 0, 0, 0, 0]",N56061-0
1,22,U80798,2019-11-09 14:23:59,"[N52946, 0, 0, 0, 0]",
2,32,U6744,2019-11-12 12:21:35,"[N9824, 0, 0, 0, 0]",
3,35,U78244,2019-11-12 12:58:56,"[N44956, 0, 0, 0, 0]",
4,54,U2579,2019-11-13 06:58:59,"[0, 0, 0, 0, 0]",N59088-0
...,...,...,...,...,...
10085,156909,U58715,2019-11-11 04:20:16,"[0, 0, 0, 0, 0]",N56061-0
10086,156923,U62649,2019-11-13 07:12:20,"[0, 0, 0, 0, 0]",N59088-0
10087,156945,U31444,2019-11-13 06:34:19,"[0, 0, 0, 0, 0]",N59088-0
10088,156956,U66493,2019-11-13 12:02:07,"[N52946, 0, 0, 0, 0]",


In [287]:
## adding histroy count column
df_ai_user['History_count'] = df_ai_user['History'].apply(lambda x: len(x))
df_ai_user

,Impression ID,User ID,Time,History,Impressions,History_count
0,18,U11306,2019-11-11 04:29:18,"[0, 0, 0, 0, 0]",N56061-0,5
1,22,U80798,2019-11-09 14:23:59,"[N52946, 0, 0, 0, 0]",,5
2,32,U6744,2019-11-12 12:21:35,"[N9824, 0, 0, 0, 0]",,5
3,35,U78244,2019-11-12 12:58:56,"[N44956, 0, 0, 0, 0]",,5
4,54,U2579,2019-11-13 06:58:59,"[0, 0, 0, 0, 0]",N59088-0,5
...,...,...,...,...,...,...
10085,156909,U58715,2019-11-11 04:20:16,"[0, 0, 0, 0, 0]",N56061-0,5
10086,156923,U62649,2019-11-13 07:12:20,"[0, 0, 0, 0, 0]",N59088-0,5
10087,156945,U31444,2019-11-13 06:34:19,"[0, 0, 0, 0, 0]",N59088-0,5
10088,156956,U66493,2019-11-13 12:02:07,"[N52946, 0, 0, 0, 0]",,5


In [288]:
## Verifying padding
df_ai_user['History_count'].unique()

array([5])

In [289]:
df_ai_news


,News ID,Title,Abstract,keywords,category,cleaned_tags
0,N2804,"Google discontinues Clips, the AI-powered came...",While Google was busy showcasing its latest de...,"['latest devices yesterday', 'Google discontin...",Incidents & Safety,"[policy/Regulation, topic/Safety, geo/US]"
1,N96829,AI can help doctors spot brain hemorrhages faster,AI is already capable of discovering medical c...,"['discovering medical conditions', 'doctors sp...",Research,"[domain/Search, event/GoogleIO, topic/Multimod..."
2,N118770,Pentagon's draft AI ethics guidelines fight bi...,Tech companies might have trouble establishing...,"['rogue machines Tech', 'machines Tech compani...",Policy & Regulation,"[org/Google, domain/Technology, event/GoogleIO]"
3,N110386,Google Says New AI-Powered Search Update Is 'H...,Google is injecting its search engine with new...,"['Huge Step Forward', 'AI-Powered Search Updat...",Technology & Product,"[domain/Healthcare, topic/Multimodal, topic/RA..."
4,N83002,Sonar drone helps find a WWII Japanese aircraf...,The late Paul Allen's underwater robotics are ...,"['Paul Allen underwater', 'late Paul Allen', '...",Incidents & Safety,"[domain/Transportation, topic/Robotics, geo/US..."
...,...,...,...,...,...,...
142,N51029,Big Tech tackles the drug diversion opioid cri...,"Some 10% of pharmacists, nurses, and anesthesi...","['Big Tech tackles', 'diversion opioid crisis'...",Misc,"[domain/Healthcare, org/Google, org/Microsoft,..."
143,N105010,Alphabet's rebooted robotics program starts wi...,For all the advances made by robot companies l...,"['performing assistive tasks', 'Boston Dynamic...",Misc,"[domain/Robotics, org/Google, topic/Agents, ev..."
144,N78685,2 Simple Steps to Withstand Google's Constant ...,You must do the best job on the internet at de...,"['Withstand Google Constant', 'Google Constant...",Misc,"[domain/Fintech, biz/Funding, event/GoogleIO, ..."
145,N86909,Wild Animation Shows All the Multi-Planet Syst...,The tiny dots represent all the multi-planet s...,"['Kepler Space Telescope', 'Systems Kepler Dis...",Society & Culture,"[domain/Search, policy/Regulation, topic/RAG]"


In [290]:
df_ai_user.iloc[0]

Impression ID                     18
User ID                       U11306
Time             2019-11-11 04:29:18
History              [0, 0, 0, 0, 0]
Impressions                 N56061-0
History_count                      5
Name: 0, dtype: object

In [292]:
df_ai_news.columns

Index(['News ID', 'Title', 'Abstract', 'keywords', 'category', 'cleaned_tags'], dtype='object')

In [293]:
news_features = df_ai_news[['News ID', 'category', 'Abstract', 'Title', 'cleaned_tags']].set_index('News ID')
def enrich_history(hist):
    return [news_features.loc[nid].to_dict() if nid in news_features.index else 
            {'category': 'unk', 'Abstract': '', 'Title': '', 'cleaned_tags': []} for nid in hist]
df_ai_user['History_features'] = df_ai_user['History'].apply(enrich_history)

In [ ]:
df_ai_news.to_csv('./data/ai_news_final.csv', index=False)
df_ai_user.to_csv('./data/ai_user_final.csv', index=False)